In [18]:
# Import Python Libraries

# For HTML parsing
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

# For website connections
import requests 

# To prevent overwhelming the server between connections
from time import sleep 

# Display the progress bar
from tqdm.notebook import tqdm

# For data wrangling
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# For creating plots
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [19]:
#Pull request from 99.co website
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
url = 'https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=2'
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")

In [20]:
def get_page(page, town = 'bukit_panjang'):
    #default_town is 'bukit_panjang'
    url = f'https://www.99.co/singapore/sale?query_ids=ht{town}&query_limit=radius&query_type=hdb_town&page_num={page}'
    #url = 'https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=2'
    print(url)
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
        
    return page_soup

In [21]:
#data processing for each page
def data_process(page_soup):
    prices = page_soup.findAll("p","_2sIc2 JlU_W _2rhE-")
    rooms = page_soup.findAll("p",'dniCg _1LPAx _2rhE- _1c-pJ')
    i = 0
    lol = []
    for price in prices:
        compiler = []
        compiler.append(price.getText())
        compiler.append(rooms[i].getText())
        compiler.append(rooms[i+1].getText())
        lol.append(compiler)
        i = i+2
    df = pd.DataFrame(lol, columns=["prices","type","year"])
    return df


In [22]:
#Main execution
df = pd.DataFrame()
max_entry = page_soup.find("p","dniCg GbDXT _2rhE-").getText()
max_entry = int(re.findall(r'\d+', max_entry)[0])
last_entry = 0 

for i in range(max_entry//35):
    page_soup = get_page(i+1)
    page_list = data_process(page_soup)
    df=df.append(page_list, ignore_index=True)
    last_entry = last_entry+page_list.shape[0]
    print(f'{last_entry} out of {max_entry} extracted')
    if last_entry>max_entry:
        print('exceed')
        break

#df cleaning

https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=1
35 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=2
70 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=3
105 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=4
140 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=5
175 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=6
210 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query_limit=radius&query_type=hdb_town&page_num=7
245 out of 679 extracted
https://www.99.co/singapore/sale?query_ids=htbukit_panjang&query

In [23]:
df.head()


,prices,type,year
0,"$800,000",HDB (EA),1989
1,"$4,500,000",Bungalow,1998 · 99 years
2,"$480,000",HDB (3RM),2002 · 99 years
3,"$3,900,000",Terraced House,999 years
4,"$599,999",HDB (5I),1998


In [24]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prices  630 non-null    object
 1   type    630 non-null    object
 2   year    630 non-null    object
dtypes: object(3)
memory usage: 14.9+ KB


,prices,type,year
0,"$490,000",HDB (4A),2015 · 99 years
1,"$660,000",HDB (5I),1998
2,"$425,000",HDB (4S),1988
3,"$660,000",HDB (5A),2016 · 99 years
4,"$838,000",HDB (EA),2001
